# Linear Models

The following are a set of methods intended for regression in which the target value is expected to be a linear combination of the features. In mathematical notation, if $\hat{y}$
is the predicted value.

$$
\hat{y}(w, x) = w_0 + w_1 x_1 + \dots + w_p x_p
$$


Across the module, we designate the vector $\mathbf{w} = (w_1, \dots, w_p)$ as ``coef_`` and $w_0$ as ``intercept_``.


## Ordinary Least Squares

`LinearRegression` fits a linear model with coefficients $\mathbf{w} = (w_1, \dots, w_p)$ to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation. Mathematically it solves a problem of the form


$\min_{w} \| X w - y \|_2^2$


<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ols_001.png" width=400>

`LinearRegression` will take in its fit method arrays X, y and will store the coefficients $\mathbf{w}$ of the linear model in its coef_ member:

In [2]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

LinearRegression()

In [3]:
reg.coef_

array([0.5, 0.5])

The coefficient estimates for Ordinary Least Squares rely on the
independence of the features. When features are correlated and the
columns of the design matrix $\mathbf{X}$ have an approximately linear
dependence, the design matrix becomes close to singular
and as a result, the least-squares estimate becomes highly sensitive
to random errors in the observed target, producing a large
variance. This situation of *multicollinearity* can arise, for
example, when data are collected without an experimental design.


---------
For Example

see Linear_Regression_Example.ipynb 

### Non-Negative Least Squares

It is possible to constrain all the coefficients to be non-negative, which may be useful when they represent some physical or naturally non-negative quantities (e.g., frequency counts or prices of goods). LinearRegression accepts a boolean `positive` parameter: when set to `True` Non-Negative Least Squares are then applied.

---------
For Example


Non_Negative_Least_Squares_Example.ipynb

### Ordinary Least Squares Complexity

The least squares solution is computed using the singular value
decomposition of X. If X is a matrix of shape (n_samples, n_features)
this method has a cost of
$O(n_{\text{samples}} n_{\text{features}}^2)$, assuming that
$n_{\text{samples}} \geq n_{\text{features}}$.


## Ridge regression and classification

### Regression

Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of the coefficients. The ridge coefficients minimize a penalized residual sum of squares:

$$
\min_{w} \| X w - y \|_2^2 + \alpha \|w\|_2^2
$$


The complexity parameter $\alpha \geq 0$ controls the amount
of shrinkage: the larger the value of $\alpha$, the greater the amount
of shrinkage and thus the coefficients become more robust to collinearity.


As with other linear models, Ridge will take in its `fit` method arrays `X`, `y` and will store the coefficients of the linear model in its `coef_` member:

In [4]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit([[0, 0], [0, 0], [1, 1]], [0, .1, 1])

Ridge(alpha=0.5)

In [5]:
reg.coef_

array([0.34545455, 0.34545455])

In [6]:
reg.intercept_

np.float64(0.13636363636363638)

Note that the class Ridge allows for the user to specify that the solver be automatically chosen by setting `solver="auto"`. When this option is specified, Ridge will choose between the `"lbfgs"`, `"cholesky"`, and `"sparse_cg"` solvers. Ridge will begin checking the conditions shown in the following table from top to bottom. If the condition is true, the corresponding solver is chosen.

**Solver** | Condition
------------ | -------------
‘lbfgs’ | The `positive=True` option is specified.
‘cholesky’ | The input array X is not sparse.
‘sparse_cg’ | None of the above conditions are fulfilled.

### Classification

The `Ridge` regressor has a classifier variant: `RidgeClassifier`. This classifier first converts binary targets to `{-1, 1}` and then treats the problem as a regression task, optimizing the same objective as above. The predicted class corresponds to the sign of the regressor’s prediction. For multiclass classification, the problem is treated as multi-output regression, and the predicted class corresponds to the output with the highest value.

It might seem questionable to use a (penalized) Least Squares loss to fit a classification model instead of the more traditional logistic or hinge losses. However, in practice, all those models can lead to similar cross-validation scores in terms of accuracy or precision/recall, while the penalized least squares loss used by the `RidgeClassifier` allows for a very different choice of the numerical solvers with distinct computational performance profiles.

The `RidgeClassifier` can be significantly faster than e.g. `LogisticRegression` with a high number of classes because it can compute the projection matrix $(X^T X)^{-1} X^T$ only once.

This classifier is sometimes referred to as a **Least Squares Support Vector Machines** with a linear kernel.

-------------------
For Example 

Plot_Ridge.ipynb


### Ridge Complexity

This method has the same order of complexity as Ordinary Least Squares.

### Setting the regularization parameter: leave-one-out Cross-Validation

`RidgeCV` and `RidgeClassifierCV` implement ridge regression/classification with built-in cross-validation of the alpha parameter. They work in the same way as `GridSearchCV` except that it defaults to efficient Leave-One-Out `cross-validation`. When using the default `cross-validation`, alpha cannot be 0 due to the formulation used to calculate Leave-One-Out error.

Usage Example:

In [ ]:
import numpy as np
from sklearn import linear_model
reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
reg.fit([[0, 0], [0, 0], [1, 1]], [0, .1, 1])
reg.alpha_

Specifying the value of the `cv` attribute will trigger the use of cross-validation with `GridSearchCV`, for example `cv=10` for 10-fold cross-validation, rather than Leave-One-Out Cross-Validation.

## Lasso

The `Lasso` is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer non-zero coefficients, effectively reducing the number of features upon which the given solution is dependent. For this reason, Lasso and its variants are fundamental to the field of compressed sensing. Under certain conditions, it can recover the exact set of non-zero coefficients

Mathematically, it consists of a linear model with an added regularization term. The objective function to minimize is:

$$
\min_{w} \left( \frac{1}{2n_{\text{samples}}} ||X w - y||_2^2 + \alpha ||w||_1 \right)
$$


The lasso estimate thus solves the minimization of the least-squares penalty with $ \alpha ||w||_1 $ added, where $\alpha$ is a constant and $||w||_1$ 
is the $\ell_1$-norm of the coefficient vector.

The implementation in the class `Lasso` uses coordinate descent as the algorithm to fit the coefficients.

In [ ]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit([[0, 0], [1, 1]], [0, 1])
reg.predict([[1, 1]])

The function `lasso_path` is useful for lower-level tasks, as it computes the coefficients along the full path of possible values.

---------------
Example 

Compressive_Sensing_Tomography_Reconstruction_with_L1_Prior.ipynb

### Using Cross-Validation

scikit-learn exposes objects that set the Lasso `alpha` parameter by cross-validation: `LassoCV` and `LassoLarsCV`. `LassoLarsCV` is based on the `Least Angle Regression` algorithm explained below.

For high-dimensional datasets with many collinear features, `LassoCV` is most often preferable. However, `LassoLarsCV` has the advantage of exploring more relevant values of `alpha` parameter, and if the number of samples is very small compared to the number of features, it is often faster than `LassoCV`.

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_lasso_model_selection_002.png" width="500">
<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_lasso_model_selection_003.png" width="500">

#### Information-criteria based model selection

Alternatively, the estimator `LassoLarsIC` proposes to use the Akaike information criterion (AIC) and the Bayes Information criterion (BIC). It is a computationally cheaper alternative to find the optimal value of alpha as the regularization path is computed only once instead of k+1 times when using k-fold cross-validation.

Indeed, these criteria are computed on the in-sample training set. In short, they penalize the over-optimistic scores of the different Lasso models by their flexibility (cf. to “Mathematical details” section below).

However, such criteria need a proper estimation of the degrees of freedom of the solution, are derived for large samples (asymptotic results) and assume the correct model is candidates under investigation. They also tend to break when the problem is badly conditioned (e.g. more features than samples).

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_lasso_lars_ic_001.png' width=500>

#### Comparison with the regularization parameter of SVM

The equivalence between `alpha` and the regularization parameter of SVM, `C` is given by `alpha = 1 / C` or `alpha = 1 / (n_samples * C)`, depending on the estimator and the exact objective function optimized by the model.

## Logistic Regression

The logistic regression is implemented in `LogisticRegression`. Despite its name, it is implemented as a linear model for classification rather than regression in terms of the scikit-learn/ML nomenclature. The logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a `logistic function`.

This implementation can fit binary, One-vs-Rest, or multinomial logistic regression with optional $\ell_1$,$\ell_2$ or Elastic-Net regularization.

-------------
Examples

L1_Penalty_and_Sparsity_in_Logistic_Regression.ipynb

Regularization_Path_of_L1-Logistic_Regression.ipynb

Multiclass_Sparse_Logistic_REgression_on_20newgroups.ipynb

MNIST_Classification_Using_Multinominal_Logistic_+_L1.ipynb

## Stochastic Gradient Descent - SGD

Stochastic gradient descent is a simple yet very efficient approach to fit linear models. It is particularly useful when the number of samples (and the number of features) is very large. The `partial_fit` method allows online/out-of-core learning.

The classes `SGDClassifier` and `SGDRegressor` provide functionality to fit linear models for classification and regression using different (convex) loss functions and different penalties. E.g., with `loss="log"`, `SGDClassifier` fits a logistic regression model, while with `loss="hinge"` it fits a linear support vector machine (SVM).

## Polynomial Regression: Extending Linear Mdels with Basis Functions

One common pattern within machine learning is to use linear models trained on nonlinear functions of the data. This approach maintains the generally fast performance of linear methods, while allowing them to fit a much wider range of data.

---------
Mathematical Details

For example, a simple linear regression can be extended by constructing polynomial features from the coefficients. In the standard linear regression case, you might have a model that looks like this for two-dimensional data:

$$ 
\hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2
$$

If we want to fit a paraboloid to the data instead of a plane, we can combine the features in second-order polynomials, so that the model looks like this:

$$
\hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2 + w_3 x_1 x_2 + w_4 x_1^2 + w_5 x_2^2
$$

The (sometimes surprising) observation is that this is still a linear model: to see this, imagine creating a new set of features

$$ 
z = [x_1, x_2, x_1 x_2, x_1^2, x_2^2]
$$

With this re-labeling of the data, our problem can be written

$$
\hat{y}(w, z) = w_0 + w_1 z_1 + w_2 z_2 + w_3 z_3 + w_4 z_4 + w_5 z_5
$$

We see that the resulting polynomial regression is in the same class of linear models we considered above (i.e. the model is linear in $w$) and can be solved by the same techniques. By considering linear fits within a higher-dimensional space built with these basis functions, the model has the flexibility to fit a much broader range of data.

---------
Here is an example of applying this idea to one-dimensional data, using polynomial features of varying degrees:

<img src='https://scikit-learn.org/stable/_images/sphx_glr_plot_polynomial_interpolation_001.png' width=500>

This figure is created using the `PolynomialFeatures` transformer, which transforms an input data matrix into a new data matrix of a given degree. It can be used as follows:

In [2]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
X = np.arange(6).reshape(3, 2)
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [4]:
poly = PolynomialFeatures(degree=2)
poly.fit_transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

The features of `X` have been transformed from $[x_1, x_2]$ to $[1, x_1, x_2, x_1^2, x_1 x_2, x_2^2]$ , and can now be used within any linear model.

This sort of preprocessing can be streamlined with the `Pipeline` tools. A single object representing a simple polynomial regression can be created and used as follows:

In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])
# fit to an order-3 polynomial data
x = np.arange(5)
y = 3 - 2 * x + x ** 2 - x ** 3
model = model.fit(x[:, np.newaxis], y)
model.named_steps['linear'].coef_

array([ 3., -2.,  1., -1.])

The linear model trained on polynomial features is able to exactly recover the input polynomial coefficients.

In some cases it’s not necessary to include higher powers of any single feature, but only the so-called interaction features that multiply together at most
distinct features. These can be gotten from `PolynomialFeatures` with the setting `interaction_only=True`.

For example, when dealing with boolean features, $x_i^n = x_i$
for all $n$
and is therefore useless; but $x_i x_j$
represents the conjunction of two booleans. This way, we can solve the XOR problem with a linear classifier:

In [6]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = X[:, 0] ^ X[:, 1]
y
X = PolynomialFeatures(interaction_only=True).fit_transform(X).astype(int)
X
clf = Perceptron(fit_intercept=False, max_iter=10, tol=None,
                 shuffle=False).fit(X, y)

And the classifier “predictions” are perfect:

In [7]:
clf.predict(X)

array([0, 1, 1, 0])

In [8]:
clf.score(X, y)

1.0